In [5]:
%run mathutil.ipynb

In [6]:
np.random.seed(1234) # 난수 고정
def randomize(): np.random.seed(time.timme()) 
    # randomize() 로 난수를 초기화하지 않는 이상 같은 결과를 냄

In [7]:
class Model(object): # object class 를 기반 class로 함 
    # 객체 초기화 
    def __init__(self, name, dataset):
        self.name = name
        self.dataset = dataset
        self.is_training = False
        if not hasattr(self, 'rand_std'): self.rand_std = 0.30 
            # parameter 초기화에 rand_std 필요
    # 출력문으로 객체를 출력할때의 출력 문자열 생성 방법 정의
    def __str__(self):
        return '{}/{}'.format(self.name, self.dataset)
    # 전체 과정을 실행시키는 메인 함수 
    def exec_all(self, epoch_count=10, batch_size=10, learning_rate=0.001, report=0, show_cnt = 3):
        # hyper parameter 들을 필요한 각 method 에 전달 
        self.train(epoch_count, batch_size, learning_rate, report)
        self.test()
        if show_cnt > 0: self.visualize(show_cnt)

In [8]:
class MlpModel(Model): # Model의 파생 class 
    def __init__(self, name, dataset, hconfigs):
        super(MlpModel, self).__init__(name, dataset)
            # super를 이용해 Model class를 찾아 그 객체 초기화 함수 호출; name, dataset 값 저장 
        self.init_parameters(hconfigs) # CNN 이 이용할 parameter 준비 

In [9]:
def mlp_init_parameters(self, hconfigs): # 기존 init_model() 이랑 거의 같음 
    self.hconfigs = hconfigs
    self.pm_hiddens = []

    prev_shape = self.dataset.input_shape 

    for hconfig in hconfigs:
        pm_hidden, prev_shape = self.alloc_layer_param(prev_shape, hconfig)
        self.pm_hiddens.append(pm_hidden)

    output_cnt = int(np.prod(self.dataset.output_shape)) # parameter 생성에 반영되는 vector 크기
    self.pm_output, _ = self.alloc_layer_param(prev_shape, output_cnt) # 입출력 vector 크기를 dataset 객체의 속성값에서 받음 
        
def mlp_alloc_layer_param(self, input_shape, hconfig):
    input_cnt = np.prod(input_shape)
    output_cnt = hconfig
    weight, bias = self.alloc_param_pair([input_cnt, output_cnt])

    return {'w':weight, 'b':bias}, output_cnt

def mlp_alloc_param_pair(self, shape):
    weight = np.random.normal(0, self.rand_std, shape)
    bias = np.zeros([shape[-1]])
    return weight, bias

# function을 MlpModel의 method로 등록하기 
MlpModel.init_parameters = mlp_init_parameters 
# init_parameter 는 layer 하나의 parameter를 생성할때마다 alloc_layer_param()을 부름
MlpModel.alloc_layer_param = mlp_alloc_layer_param
# alloc_layer_param 은 alloc_param_pair 를 부름
MlpModel.alloc_param_pair = mlp_alloc_param_pair

In [10]:
def mlp_model_train(self, epoch_count=10, batch_size=10, \
                    learning_rate=0.001, report=0):
    # Model.exec_all() 이 학습을 실행할 때 호출 
    self.learning_rate = learning_rate
    
    batch_count = int(self.dataset.train_count / batch_size)
    time1 = time2 = int(time.time())
    if report != 0:
        print('Model {} train started:'.format(self.name))

    for epoch in range(epoch_count):
        costs = []
        accs = []
        self.dataset.shuffle_train_data(batch_size*batch_count) # data shuffle 
        for n in range(batch_count):
            trX, trY = self.dataset.get_train_data(batch_size, n)
            cost, acc = self.train_step(trX, trY)
            costs.append(cost)
            accs.append(acc)

        if report > 0 and (epoch+1) % report == 0:
            vaX, vaY = self.dataset.get_validate_data(100)
                # 그때그때 새로 받아오는 제한된 수의 validation data 이용 
            acc = self.eval_accuracy(vaX, vaY)
            time3 = int(time.time())
            tm1, tm2 = time3-time2, time3-time1
            self.dataset.train_prt_result(epoch+1, costs, accs, acc, tm1, tm2) # validation 단계 출력 
            time2 = time3

    tm_total = int(time.time()) - time1
    print('Model {} train ended in {} secs:'.format(self.name, tm_total))
    
MlpModel.train = mlp_model_train

In [11]:
def mlp_model_test(self):
    teX, teY = self.dataset.get_test_data()
    time1 = int(time.time())
    acc = self.eval_accuracy(teX, teY)
    time2 = int(time.time())
    self.dataset.test_prt_result(self.name, acc, time2-time1) # 구체적인 출력 

MlpModel.test = mlp_model_test

In [12]:
def mlp_model_visualize(self, num):
    print('Model {} Visualization'.format(self.name))
    deX, deY = self.dataset.get_visualize_data(num) # dataset으로부터 시각화용 데이터 얻어오기 
    est = self.get_estimate(deX) # 신경망 추정 정보 얻기
    self.dataset.visualize(deX, est, deY) # 시각화 출력 생성 

MlpModel.visualize = mlp_model_visualize

In [13]:
def mlp_train_step(self, x, y): # 기존의 run_train() 함수 
    self.is_training = True # is_training 의 plug
    
    output, aux_nn = self.forward_neuralnet(x)
    loss, aux_pp = self.forward_postproc(output, y)
    accuracy = self.eval_accuracy(x, y, output)
    
    G_loss = 1.0
    G_output = self.backprop_postproc(G_loss, aux_pp)
    self.backprop_neuralnet(G_output, aux_nn)

    self.is_training = False # is_training 의 plug
        # validation은 is_training의 plug가 꺼진 상태에서 수행되어야 함 
    return loss, accuracy

MlpModel.train_step = mlp_train_step

In [14]:
def mlp_forward_neuralnet(self, x): # training, test, visualize 에서 이용 
    hidden = x
    aux_layers = []
    for n, hconfig in enumerate(self.hconfigs): # hconfigs; from MlpModel.init_parameters()
        hidden, aux = self.forward_layer(hidden, hconfig, self.pm_hiddens[n])
        aux_layers.append(aux)
    output, aux_out = self.forward_layer(hidden, None, self.pm_output)
        # hconfig 대신 None 전달; output layer 임을 표현 
    return output, [aux_out, aux_layers]

def mlp_backprop_neuralnet(self, G_output, aux):
    aux_out, aux_layers = aux # forward 에서의 전달을 그대로 받음 
    G_hidden = self.backprop_layer(G_output, None, self.pm_output, aux_out)
    
    for n in reversed(range(len(self.hconfigs))):
        hconfig, pm, aux = self.hconfigs[n], self.pm_hiddens[n], aux_layers[n]
        G_hidden = self.backprop_layer(G_hidden, hconfig, pm, aux)
    return G_hidden

MlpModel.forward_neuralnet = mlp_forward_neuralnet
MlpModel.backprop_neuralnet = mlp_backprop_neuralnet

In [19]:
def mlp_forward_layer(self, x, hconfig, pm):
    y = np.matmul(x, pm['w']) + pm['b'] # 선형 연산
    if hconfig is not None: y = relu(y) # 비선형 활성화 함수
    return y, [x,y]

def mlp_backprop_layer(self, G_y, hconfig, pm, aux):
    x, y = aux
    
    if hconfig is not None: G_y = relu_derv(y) * G_y # 비선형 활성화 함수 
    
    # 선형 연산의 backprop_
    g_y_weight = x.transpose()
    g_y_input = pm['w'].transpose()
    G_weight = np.matmul(g_y_weight, G_y)
    G_bias = np.sum(G_y, axis=0)
    G_input = np.matmul(G_y, g_y_input)
    pm['w'] -= self.learning_rate * G_weight
    pm['b'] -= self.learning_rate * G_bias

    return G_input

MlpModel.forward_layer = mlp_forward_layer
MlpModel.backprop_layer = mlp_backprop_layer

In [15]:
def mlp_forward_postproc(self, output, y):
    loss, aux_loss = self.dataset.forward_postproc(output, y)
        # dataset.forward_postproc() 으로 손실 함숫값 계산 
    extra, aux_extra = self.forward_extra_cost(y)
        # loss_extra(추가적인 손실성분), aux_extra(역전파용 보조정보) 보고 
    return loss + extra, [aux_loss, aux_extra]

def mlp_forward_extra_cost(self, y):
    return 0, None # 계산에 아무런 regularization 장치가 도입되지 않아, 계산에 반영될 추가 손실 성분 없음 

MlpModel.forward_postproc = mlp_forward_postproc
MlpModel.forward_extra_cost = mlp_forward_extra_cost

In [16]:
def mlp_backprop_postproc(self, G_loss, aux): # forward_extra_cost() 와 짝을 이룸 ; 
    aux_loss, aux_extra = aux
    self.backprop_extra_cost(G_loss, aux_extra) 
    G_output = self.dataset.backprop_postproc(G_loss, aux_loss)
    return G_output

def mlp_backprop_extra_cost(self, G_loss, aux):
    pass # 지금 당장은 빈 함수로 정의 

MlpModel.backprop_postproc = mlp_backprop_postproc
MlpModel.backprop_extra_cost = mlp_backprop_extra_cost

In [17]:
def mlp_eval_accuracy(self, x, y, output=None): # 정확도 계산 
    if output is None:
        output, _ = self.forward_neuralnet(x)
    accuracy = self.dataset.eval_accuracy(x, y, output)
    return accuracy

MlpModel.eval_accuracy = mlp_eval_accuracy

In [18]:
def mlp_get_estimate(self, x):
    output, _ = self.forward_neuralnet(x)
        # 값 지정이 누락된 경우 forward_neuralnet() 호출해 직접 순전파 처리 수행 
    estimate = self.dataset.get_estimate(output)
    return estimate

MlpModel.get_estimate = mlp_get_estimate